In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Table 기반 답변 생성하기

##### Pandas DataFrame을 활용하여 테이블 기반 답변 생성하기

In [1]:
import pandas as pd

path = '../Assets/data/강의리스트.csv'

df = pd.read_csv(path)
df.head()

,강의 제목,강의 내용,강의 추천 대상,강의 가격,강사진
0,데이터 분석 입문 온라인 완주반,"데이터 분석에 대한 기본적인 이해를 바탕으로 Python을 활용하여 데이터를 수집,...",데이터 분석 입문자,₩200000,"John Doe, Jane Smith"
1,웹 프로그래밍 올인원 패키지 Online,"HTML, CSS, JavaScript부터 시작하여 백엔드 개발에 필요한 Node....",웹 개발 입문자,₩250000,"Emily Brown, Alex Johnson"
2,딥러닝을 활용한 자연어 처리 입문,"자연어 처리(NLP)의 기초 이론부터 실제 딥러닝 모델을 활용한 텍스트 분류, 감정...",자연어 처리 입문자,₩300000,"David Lee, Sarah Miller"
3,파이썬을 활용한 웹 데이터 수집,웹 스크래핑과 API를 이용하여 다양한 웹 데이터를 수집하는 방법을 다룹니다. 이 ...,웹 데이터 수집 입문자,₩180000,"Michael Clark, Jennifer White"
4,클라우드 네이티브 애플리케이션 개발,클라우드 기술을 기반으로 한 모던한 애플리케이션 개발 방법을 다룹니다. 이 강의를 ...,클라우드 개발 입문자,₩280000,"Daniel Wilson, Jessica Taylor"


In [2]:
markdown_df = df.to_markdown()
print(markdown_df)

|    | 강의 제목                           | 강의 내용                                                                                                                                                                                                                                                          | 강의 추천 대상        | 강의 가격   | 강사진                        |
|---:|:------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------|:------------|:------------------------------|
|  0 | 데이터 분석 입문 온라인 완주반      | 데이터 분석에 대한 기본적인 이해를 바탕으로 Python을 활용하여 데이터를 수집, 정제, 분석하는 방법을 학습합니다. 이 강의는 데이터 분석을 처음 시작하는 분들을 위해 기초적인 내용부터 차근차근 알려주며, 실습을 통해 실제 데이터를 다루는 경험을 제공합니다.          | 데이터 분석 입문자    | ₩200000     | John Doe, Jane Smith          |
|  1 | 웹 프로그래밍 올인

In [3]:
from string import Template

query = "데이터 분석 입문 온라인 완주반의 가격과 수업 내용을 알 수 있나요?"


system_prompt = Template("""
너는 온라인 강의에 대해 설명을 제공하는 인공지능 챗봇이야.
강의 설명 테이블을 참고하여서 사용자의 질문에 친절하게 답변해줘.

강의 설명 테이블
$lecture_table
""")

prompt = Template("""
다음 사용자의 요청에 대한 답변을 해주고, 답변에 근거가 되는 테이블 행도 같이 markdown 형식으로 출력해줘

요청: $query
""")

print(system_prompt.substitute(lecture_table=markdown_df))
print(prompt.substitute(query=query))


너는 온라인 강의에 대해 설명을 제공하는 인공지능 챗봇이야.
강의 설명 테이블을 참고하여서 사용자의 질문에 친절하게 답변해줘.

강의 설명 테이블
|    | 강의 제목                           | 강의 내용                                                                                                                                                                                                                                                          | 강의 추천 대상        | 강의 가격   | 강사진                        |
|---:|:------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------|:------------|:------------------------------|
|  0 | 데이터 분석 입문 온라인 완주반      | 데이터 분석에 대한 기본적인 이해를 바탕으로 Python을 활용하여 데이터를 수집, 정제, 분석하는 방법을 학습합니다. 이 강의는 데이터 분석을 처음 시작하는 분들을 위해 기초적인 내용부터 차근차근 알려주며, 실습을 통해 실제 데이터를 다루는 경험을 제공합니다.        

In [5]:
from openai import OpenAI

client = OpenAI()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt.substitute(lecture_table=markdown_df),
        },
        {
            "role": "user",
            "content": prompt.substitute(query=query),
        }
    ],
    model="gpt-4o-mini",
)

print(chat_completion.choices[0].message.content)

데이터 분석 입문 온라인 완주반의 가격은 ₩200,000이며, 수업 내용은 다음과 같습니다:

이 강의는 데이터 분석에 대한 기본적인 이해를 바탕으로 Python을 활용하여 데이터를 수집, 정제, 분석하는 방법을 학습합니다. 데이터 분석을 처음 시작하는 분들을 위해 기초적인 내용부터 차근차근 알려주며, 실습을 통해 실제 데이터를 다루는 경험을 제공합니다.

|    | 강의 제목                           | 강의 내용                                                                                                                                                                                                                                                          | 강의 추천 대상        | 강의 가격   | 강사진                        |
|---:|:------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------|:------------|:------------------------------|
|  0 | 데이터 분석 입문 온라인 완주반      | 데이터 분석에 대한 기본적인 이해를 바탕으로 Python을 활용하

In [6]:
def create_answer(query):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": system_prompt.substitute(lecture_table=markdown_df),
          },
          {
              "role": "user",
              "content": prompt.substitute(query=query),
          }
      ],
      model="gpt-4o-mini",
  )

  return chat_completion.choices[0].message.content


query_list = [
    '전체 강의의 수강료 평균은 얼마인가요?',
    '가장 비싼 강의와 가장 저렴한 강의는 무엇인가요?'
]

for q in query_list:
  print(f"Q: {q}")
  print(f"A: {create_answer(q)}")
  print("\n===============================\n")

Q: 전체 강의의 수강료 평균은 얼마인가요?
A: 전체 강의의 수강료 평균을 계산하려면 각 강의의 수강료를 더한 후 강의 수로 나누면 됩니다.

강의 수는 5개이고, 각 강의의 수강료는 다음과 같습니다:
- 데이터 분석 입문 온라인 완주반: ₩200,000
- 웹 프로그래밍 올인원 패키지 Online: ₩250,000
- 딥러닝을 활용한 자연어 처리 입문: ₩300,000
- 파이썬을 활용한 웹 데이터 수집: ₩180,000
- 클라우드 네이티브 애플리케이션 개발: ₩280,000

수강료 총합: ₩200,000 + ₩250,000 + ₩300,000 + ₩180,000 + ₩280,000 = ₩1,210,000

따라서 평균 수강료는 ₩1,210,000 / 5 = ₩242,000입니다.

따라서, 전체 강의의 수강료 평균은 ₩242,000입니다.

여기 관련된 강의 설명 테이블을 출력합니다:

|    | 강의 제목                           | 강의 내용                                                                                                                                                                                                                                                          | 강의 추천 대상        | 강의 가격   | 강사진                        |
|---:|:------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
# 평균 구하기
print("평균 가격", df['강의 가격'].str.replace('₩', '').astype(int).mean())

# 최대값 구하기
print("최고 가격", df['강의 가격'].str.replace('₩', '').astype(int).max())
print("최저 가격", df['강의 가격'].str.replace('₩', '').astype(int).min())

평균 가격 242000.0
최고 가격 300000
최저 가격 180000
